In [1]:
# libraries
import datetime

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers

In [2]:
# download dataset from the google api
measurements = pd.read_csv('http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv', header=None)

In [3]:
# extract labels from dataset
x = measurements.iloc[:, :-1]
y = measurements.iloc[:, -1]
# change x from dataframe to an array for ResNet input
x = np.array(x[:])

# split dataset into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=120)

# reshape data again for ResNet Input
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

# get dummy variables for the class labels
y_train1 = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

# graph abnormal and normal points to visualize differences.
y_train2 = y_train.to_numpy(y_train)

# checking if both shapes are correct for ResNet input
print(x_train.shape)
print(x_test.shape)

print(y_train1.shape)
print(y_test.shape)

(3498, 140, 1)
(1500, 140, 1)
(3498, 2)
(1500, 2)


In [4]:
# initialize some training variables
num_observations, feature_set, resnet_depth = x_train.shape
num_classes = 2

In [15]:
# Build ResNet Model, based off a CNN, but deeper and an extra layer (residual)
model_input = layers.Input(shape=(feature_set, resnet_depth))
convo_layer = layers.Conv1D(filters=64, kernel_size=6, strides=1)(model_input)
convo_layer_2 = layers.Conv1D(filters=64, kernel_size=6, strides=1, padding='same')(convo_layer)
activation_1 = layers.Activation("relu")(convo_layer_2)
convo_layer_3 = layers.Conv1D(filters=64, kernel_size=6, strides=1, padding='same')(activation_1)
residual_1 = layers.Add()([convo_layer_3, convo_layer])
activation_2 = layers.Activation("relu")(residual_1)
pool_1 = layers.MaxPooling1D(pool_size=6, strides=2)(activation_2)
convo_layer_4 = layers.Conv1D(filters=64, kernel_size=6, strides=1, padding='same')(pool_1)
activation_3 = layers.Activation("relu")(convo_layer_4)
convo_layer_5 = layers.Conv1D(filters=64, kernel_size=6, strides=1, padding='same')(activation_3)
residual_2 = layers.Add()([convo_layer_5, pool_1])
activation_4 = layers.Activation("relu")(residual_1)
pool_2 = layers.MaxPooling1D(pool_size=6, strides=2)(activation_4)
convo_layer_6 = layers.Conv1D(filters=64, kernel_size=6, strides=1, padding='same')(pool_2)
activation_5 = layers.Activation("relu")(convo_layer_6)
convo_layer_7 = layers.Conv1D(filters=64, kernel_size=6, strides=1, padding='same')(activation_5)
residual_3 = layers.Add()([convo_layer_7, pool_2])
activation_6 = layers.Activation("relu")(residual_3)
pool_3 = layers.MaxPooling1D(pool_size=6, strides=2)(activation_6)
convo_layer_8 = layers.Conv1D(filters=64, kernel_size=6, strides=1, padding='same')(pool_3)
activation_7 = layers.Activation("relu")(convo_layer_8)
convo_layer_9 = layers.Conv1D(filters=64, kernel_size=6, strides=1, padding='same')(activation_7)
residual_4 = layers.Add()([convo_layer_9, pool_3])
activation_8 = layers.Activation("relu")(residual_4)
pool_4 = layers.MaxPooling1D(pool_size=6, strides=2)(activation_8)
convo_layer_10 = layers.Conv1D(filters=64, kernel_size=6, strides=1, padding='same')(pool_4)
activation_9 = layers.Activation("relu")(convo_layer_10)
convo_layer_11 = layers.Conv1D(filters=64, kernel_size=6, strides=1, padding='same')(activation_9)
residual_5 = layers.Add()([convo_layer_11, pool_4])
activation_10 = layers.Activation("relu")(residual_5)
pool_5 = layers.MaxPooling1D(pool_size=6, strides=2)(activation_10)
flat_1 = layers.Flatten()(pool_5)
fcl_1 = layers.Dense(32)(flat_1)
activation_11 = layers.Activation("relu")(fcl_1)
fcl_2 = layers.Dense(32)(activation_11)
output_layer = layers.Dense(num_classes, activation="softmax")(fcl_2)

model = models.Model(inputs=model_input, outputs=output_layer)

model.summary()  # summarize ResNet model

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 140, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_62 (Conv1D)              (None, 135, 64)      448         input_7[0][0]                    
__________________________________________________________________________________________________
conv1d_63 (Conv1D)              (None, 135, 64)      24640       conv1d_62[0][0]                  
__________________________________________________________________________________________________
activation_58 (Activation)      (None, 135, 64)      0           conv1d_63[0][0]                  
____________________________________________________________________________________________

In [16]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'Precision', 'Recall'])

# log model information
log_dir = "logs1/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_70_30"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [17]:
# initialize training variables
batch_size = 100
num_epochs = 15

In [19]:
# fit model, don't forget the callback for information logging
model.fit(x_train, y_train1, epochs=num_epochs, batch_size=batch_size, callbacks=[tensorboard_callback])

# get predictions
y_pred = model.predict(x_test)

# reshape arrays so we are able to get the different metrics
y_pred = np.nanargmax(y_pred, axis=1)
# y_test = np.nanargmax(y_test, axis=1)

# print accuracy metrics
print("F1 Score:  ", f1_score(y_test, y_pred, average="macro"))
print("Precision: ", precision_score(y_test, y_pred, average="macro"))
print("Recall:    ", recall_score(y_test, y_pred, average="macro"))
print("Accuracy:  ", np.nanmean((y_test == y_pred) * 1.0))

Epoch 1/15
35/35 [==============================] - 6s 178ms/step - loss: 0.0196 - accuracy: 0.9946 - precision: 0.9946 - recall: 0.9946
Epoch 2/15
35/35 [==============================] - 6s 178ms/step - loss: 0.0199 - accuracy: 0.9943 - precision: 0.9943 - recall: 0.9943
Epoch 3/15
35/35 [==============================] - 6s 177ms/step - loss: 0.0137 - accuracy: 0.9966 - precision: 0.9966 - recall: 0.9966
Epoch 4/15
35/35 [==============================] - 6s 179ms/step - loss: 0.0137 - accuracy: 0.9960 - precision: 0.9960 - recall: 0.9960
Epoch 5/15
35/35 [==============================] - 6s 179ms/step - loss: 0.0103 - accuracy: 0.9980 - precision: 0.9980 - recall: 0.9980
Epoch 6/15
35/35 [==============================] - 6s 179ms/step - loss: 0.0050 - accuracy: 0.9991 - precision: 0.9991 - recall: 0.9991
Epoch 7/15
35/35 [==============================] - 6s 179ms/step - loss: 0.0038 - accuracy: 0.9997 - precision: 0.9997 - recall: 0.9997
Epoch 8/15
35/35 [=======================